In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,  BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from functools import reduce
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import math

In [2]:
diabetes_data = pd.read_csv("../data/diabetes/diabetes_preprocessed.csv", index_col = 0)
diabetes_data.head()

,Age,Polyuria,Polydipsia,Sudden_Weight_Loss,Weakness,Polyphagia,Genital_thrush,Visual_Blurring,Itching,Irritability,Delayed_Healing,Partial_Paresis,Muscle_Stiffness,Alopecia,Obesity,Class,Gender_Female,Gender_Male
0,40,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1,0,1
1,58,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1
2,41,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1,0,1
3,45,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1,0,1
4,60,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1


# Задание 1

In [3]:
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]

df = pd.DataFrame(data, columns=['weather', 'stroll'])
df

,weather,stroll
0,солнечно,True
1,снег,False
2,облачно,False
3,дождь,False
4,солнечно,True
5,снег,False
6,облачно,True
7,снег,False
8,солнечно,False
9,облачно,True


Вероятность отправится на прогулку если пойдет дождь

In [4]:
pRain = len(df.loc[df.weather == 'дождь']) / len(df)
pStroll = len(df.loc[df.stroll == True]) / len(df)
pRainStroll = len(df.loc[(df.stroll == True) & (df.weather == 'дождь')]) / len(df.loc[df.stroll == True])

pStrollRain = pStroll * pRainStroll / pRain
pStrollRain

0.3333333333333333

# Задание 2

In [5]:
y = diabetes_data["Class"]
diabetes_data.drop('Class', axis =1, inplace = True)
X = diabetes_data

In [6]:
likelyhoods = {} #возможность
class_p = {} # априорная вероятность класса
prior_p = {} # априорная вероятность предиктора

In [7]:
def calculate_class_p(y):
    classes, count = np.unique(y, return_counts = True)
    result = count / len(y)
    return result
calculate_class_p(y)

array([0.38461538, 0.61538462])

In [8]:
def calculate_likelihood(feature, y):
    return [ ((y[feature == 1] == 1).sum() / (y == 1).sum(), 
    (y[feature == 0] == 1).sum() / (y == 1).sum()),
    ((y[feature == 1] == 0).sum() / (y == 0).sum(),
    (y[feature == 0] == 0).sum() / (y == 0).sum())]


In [33]:
def calculate_numeric(feature, y):
    return (feature[y[y == 1].index].mean(), feature[y[y == 1].index].std())

In [34]:
def calculate_numeric(a, b, val):
    return 1 / ((2*np.pi * a)**0.5) * math.exp(-1 * (val - b) ** 2 / (2 * a**2))

In [39]:
class NaiveBayes:
    
    def fit(self, X, y, numerics):
        classes, count = np.unique(y, return_counts = True)
        class_p = count[classes == 1] / len(y)
        self._classes = classes
        self._class_p = class_p
        self._numerics_indexes = numerics
        probs = []
        for x in X:
            probs.append(calculate_likelihood(X[x], y))
        self._probs = probs
        numerics = []
        for n in numerics:
            numerics.append(calculate_numeric(X[n], y))
        self._numerics = numerics
        
    def predict(self, X):
        result = []
        for i in range(0, X.shape[0]):
            probs_yes = []
            probs_no = []
            for j in range(0, X.shape[1]):
                if(j in self._numerics_indexes):
                    continue
                if(X.iat[i,j] == 1):
                    probs_yes.append(self._probs[j][0][0])
                    probs_no.append(self._probs[j][1][0])
                elif (X.iat[i,j] == 0):
                    probs_yes.append(self._probs[j][0][1])
                    probs_no.append(self._probs[j][1][1])
            
            for n in range(0, len(self._numerics)):
                probs_yes.append(calculate_numeric(self._numeric[n][1], self._numeric[n][0], X[self._numerics_indexes[n]]))
            
            prob_yes = reduce(lambda x, y: x * y, probs_yes)
            prob_no = reduce(lambda x, y: x * y, probs_no)
            n_prob_yes = prob_yes / (prob_yes + prob_no)
            n_prob_no = prob_no / (prob_yes + prob_no)
            result.append(round(n_prob_yes))
        return result
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
nb = NaiveBayes()
nb.fit(X_train, y_train, ['Age'])
y_pred = nb.predict(X_test)



In [40]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[47  3]
 [10 70]]
              precision    recall  f1-score   support

           0       0.82      0.94      0.88        50
           1       0.96      0.88      0.92        80

    accuracy                           0.90       130
   macro avg       0.89      0.91      0.90       130
weighted avg       0.91      0.90      0.90       130

